In [1]:
import httpx
from fasthtml.common import show
from bs4 import BeautifulSoup
from fasthtml.common import L, first, last
from fastcore.xtras import globtastic
from fastcore.parallel import parallel

In [2]:
# I just copied this from devTools -> dashboard request -> copy as cURL. You will have to replace the session_ cookie with your own.

headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'accept-language': 'en-US,en;q=0.9,es;q=0.8,ca;q=0.7,zh-TW;q=0.6,zh;q=0.5',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36',
    'sec-ch-ua': '"Google Chrome";v="131", "Chromium";v="131", "Not_A Brand";v="24"'
}

r = httpx.get('https://solveit.fast.ai/dashboard', 
              headers=headers,
              cookies={'session_': SESSION_COOKIE},
              follow_redirects=True)


In [3]:
soup = BeautifulSoup(r.text, 'html.parser')


In [4]:
links = L(soup.select('ul.uk-list li a[href]')).map(lambda x: x['href']).filter(lambda x:'youtu' in x)
links

(#12) ['https://youtu.be/4x9wtrDOXac','https://youtube.com/live/X_m7qz5LteY?feature=share','https://youtube.com/live/kEGQSamwpuU?feature=share','https://youtube.com/live/HO1gmouGEVI?feature=share','https://youtube.com/live/w_YltetZc3M?feature=share','https://youtube.com/live/83tbt5lEUgI?feature=share','https://youtube.com/live/Dnu39sQCV3k?feature=share','https://youtube.com/live/lA-KvHeFJkI?feature=share','https://youtube.com/live/ObliDyAdRx8?feature=share','https://youtube.com/live/5ZPFC_Xnz8A?feature=share','https://youtube.com/live/AvItkLCD7v8?feature=share','https://youtube.com/live/rr-59PLkIdY?feature=share']

## Download videos


In [5]:
import yt_dlp

def download_video(url, output_path='%(title)s.%(ext)s'):
    ydl_opts = {
        'format': 'best',  # best quality
        'outtmpl': output_path,  # output template
        'quiet': False,  # show progress
    }
    
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])


In [6]:

# Download a single video
# download_video('https://youtu.be/4x9wtrDOXac')


In [7]:
def download_audio(url, output_path='./audio/%(title)s.%(ext)s'):
    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
        'outtmpl': output_path,
        'quiet': False,
    }
    
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])


In [8]:
# download_audio('https://youtu.be/4x9wtrDOXac')


In [ ]:
from fastcore.parallel import parallel

# parallel(download_audio, links)

[youtube] Extracting URL: https://youtube.com/live/HO1gmouGEVI?feature=share
[youtube] HO1gmouGEVI: Downloading webpage
[youtube] Extracting URL: https://youtube.com/live/AvItkLCD7v8?feature=share
[youtube] Extracting URL: https://youtube.com/live/83tbt5lEUgI?feature=share
[youtube] AvItkLCD7v8: Downloading webpage
[youtube] 83tbt5lEUgI: Downloading webpage
[youtube] Extracting URL: https://youtube.com/live/X_m7qz5LteY?feature=share
[youtube] Extracting URL: https://youtube.com/live/5ZPFC_Xnz8A?feature=share
[youtube] X_m7qz5LteY: Downloading webpage
[youtube] Extracting URL: https://youtube.com/live/Dnu39sQCV3k?feature=share
[youtube] Extracting URL: https://youtube.com/live/kEGQSamwpuU?feature=share
[youtube] 5ZPFC_Xnz8A: Downloading webpage
[youtube] kEGQSamwpuU: Downloading webpage
[youtube] Dnu39sQCV3k: Downloading webpage
[youtube] Extracting URL: https://youtu.be/4x9wtrDOXac
[youtube] 4x9wtrDOXac: Downloading webpage
[youtube] Extracting URL: https://youtube.com/live/lA-KvHeFJkI

## Diarize audio


Unfortunately, HTTP GET method has a payload size limit, so providing the file direclty is not possible. It's possible to work around uploading it somewhere which is publicly accessible and then use a URL. Instead, we'll manually use macWhisper for now.

[See issue](https://github.com/replicate/replicate-python/issues/157)

In [11]:
import replicate

def diarize_with_replicate(filepath, num_speakers=2, language="en", prompt=""):
    input = {
        "file": open(filepath, "rb"),
        "prompt": prompt,
        "language": language,
        "num_speakers": num_speakers
    }
    
    output = replicate.run(
        "thomasmol/whisper-diarization:cbd15da9f839c5f932742f86ce7def3a03c22e2b4171d42823e83e314547003f",
        input=input
    )
    return output

# Run diarization
result = diarize_with_replicate(
    "SIWC - lesson1.mp3", 
    prompt="SolveIT lesson, coding tutorial",
    num_speakers=2
)
print(result)

ReplicateError: ReplicateError Details:
status: 413

In [12]:
# this will also fail as it uses Replicate behind the scenes
import httpx

def diarize_audio(filepath, instruction="I want to diarize this audio file of a SolveIT lesson in English with Jeremy Howard and Johnathan Whitaker speaking."):
    url = "https://web-production-1ac0.up.railway.app/diarize/chat"
    headers = {
        "accept": "application/json",
        "Authorization": f"Bearer {TOKEN}"
    }
    
    files = {
        "file": ("audio.mp3", open(filepath, "rb")),
        "instruction": (None, instruction)
    }
    
    r = httpx.post(url, headers=headers, files=files)
    return r.json()

# Assuming the first downloaded file is named by its title
# Get the first downloaded file path
# first_file = "SIWC - lesson1.mp3"  # adjust this to your actual filename
# result = diarize_audio(first_file)
# print(result)

## Produce titles

In [35]:
t1 = open('./transcripts/SIWC - lesson1.txt').read()
t1[:100]

'Before we kind of have our official start, I just wanted to do a couple of quick logistics things, w'

In [5]:
from claudette import Chat, Client, models
model = models[1]
model

'claude-3-5-sonnet-20241022'

In [6]:
chat = Chat(model, sp="You are a helpful assistant that produces titles, and summaries for SolveIT lessons.")
chat(t1)

Here's a title and summary for this lesson:

Title: "Introduction to Dialogue Engineering with Python and Advent of Code"

Summary:
This first lesson introduces the SolveIT platform and dialogue engineering approach to problem solving with AI and code. Key topics covered:

1. Platform Overview:
- Introduction to SolveIT, a custom educational platform combining code, notes, and AI assistance
- Explanation of dialogue-based problem solving approach
- Basic platform features and keyboard shortcuts

2. Problem Solving Framework:
- Introduction to Advent of Code programming challenges
- Demonstration of systematic problem solving using George Pólya's methodology
- Emphasis on small, verifiable steps and data visualization

3. Practical Example:
- Walked through solving Advent of Code 2023 Day 1 Part A
- Demonstrated iterative development process
- Showed how to combine Python coding with AI assistance

4. Best Practices:
- Importance of checking documentation
- Value of testing code in small increments
- How to effectively combine human coding skills with AI assistance

The lesson concluded with homework assignments to complete Day 1 Part B and optionally Day 2, while emphasizing the importance of following the demonstrated methodical approach to problem solving.

<details>

- id: `msg_01PF65eQeFYeQw7Aek9eUY6J`
- content: `[{'citations': None, 'text': 'Here\'s a title and summary for this lesson:\n\nTitle: "Introduction to Dialogue Engineering with Python and Advent of Code"\n\nSummary:\nThis first lesson introduces the SolveIT platform and dialogue engineering approach to problem solving with AI and code. Key topics covered:\n\n1. Platform Overview:\n- Introduction to SolveIT, a custom educational platform combining code, notes, and AI assistance\n- Explanation of dialogue-based problem solving approach\n- Basic platform features and keyboard shortcuts\n\n2. Problem Solving Framework:\n- Introduction to Advent of Code programming challenges\n- Demonstration of systematic problem solving using George Pólya\'s methodology\n- Emphasis on small, verifiable steps and data visualization\n\n3. Practical Example:\n- Walked through solving Advent of Code 2023 Day 1 Part A\n- Demonstrated iterative development process\n- Showed how to combine Python coding with AI assistance\n\n4. Best Practices:\n- Importance of checking documentation\n- Value of testing code in small increments\n- How to effectively combine human coding skills with AI assistance\n\nThe lesson concluded with homework assignments to complete Day 1 Part B and optionally Day 2, while emphasizing the importance of following the demonstrated methodical approach to problem solving.', 'type': 'text'}]`
- model: `claude-3-5-sonnet-20241022`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 19032, 'output_tokens': 274}`

</details>

In [7]:
from fastcore.basics import BasicRepr
class Summary(BasicRepr):
    "Information about a SolveIT lesson"
    def __init__(self, 
                title:str, # title of the lesson
                summary:str, # summary of the lesson
                ):
        self.title = title
        self.summary = summary

cli = Client(model)
s = cli.structured(f"You are a helpful assistant that produces titles, and summaries for SolveIT lessons. Provide information from the following SolveIT lesson transcript. {t1}", Summary)
s


[Summary(title='Introduction to SolveIT and Dialogue Engineering', summary='This introductory lesson covered several key topics:\n\n1. Platform logistics and guidelines:\n- 2GB disk quota limit for users\n- Resource usage considerations\n- Discord usage tips and organization\n\n2. Introduction to SolveIT platform:\n- Custom-built educational environment combining code, notes, and AI assistance\n- Features like persistent Python interpreter, markdown support, and variable tracking\n- Export capabilities to notebooks/Python files\n\n3. Dialogue Engineering approach:\n- New methodology for combining code and language models\n- Focus on iterative, controlled problem-solving\n- Emphasis on user control rather than AI dominance\n\n4. Practical demonstration:\n- Solved Advent of Code 2023 Day 1A problem\n- Demonstrated step-by-step problem solving approach\n- Showed how to break down problems into small, testable pieces\n- Emphasized checking data/output at each step\n\n5. Best practices:\n- 

In [18]:
def summarize_lesson(filepath):
    t = open(filepath).read()
    cli = Client(model)
    return cli.structured(f"You are a helpful assistant that produces titles, and summaries for SolveIT lessons. Provide information from the following SolveIT lesson transcript. {t}", Summary)


In [9]:
# let's get all the transcripts
ts = globtastic('./transcripts', file_glob='*.txt')
ts


(#12) ['transcripts/SIWC Lesson 8.txt','transcripts/SIWC Lesson 9.txt','transcripts/SIWC - lesson1.txt','transcripts/SIWC Lesson 11.txt','transcripts/SIWC Lesson 10.txt','transcripts/SIWC Lesson 12.txt','transcripts/SIWC Lesson 2.txt','transcripts/SIWC Lesson 3.txt','transcripts/SIWC Lesson 7.txt','transcripts/SIWC Lesson 6.txt','transcripts/SIWC Lesson 4.txt','transcripts/SIWC Lesson 5.txt']

In [ ]:
# I had some error running in parallel not sure why
# summaries = parallel(summarize_lesson, ts)

BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.

In [21]:
summaries = ts.map(summarize_lesson)
summaries

(#12) [[Summary(title='Lean Startup, Iteration Speed, and Building Better Tools with Eric Ries', summary='In this lesson, Eric Ries joins to discuss his reactions to SolveIT and shares insights from his experience developing the Lean Startup methodology. Key themes include:\n\n1. The importance of rapid iteration and feedback loops in product development\n- Smaller batch sizes and faster feedback cycles lead to better outcomes\n- The build-measure-learn feedback loop is fundamental to learning and improvement\n- Technical tools that reduce friction enable faster iteration\n\n2. The value of being your own customer\n- Building tools you want to use yourself provides immediate feedback\n- Co-evolution between developers and tools leads to better products\n- Internal usage as a key metric of success\n\n3. The role of experimentation and scientific thinking\n- Treating assumptions as hypotheses that need testing\n- The importance of defining clear expectations and measuring outcomes\n- A/B

In [28]:
zipped = L(zip(ts, summaries))
zipped


(#12) [('transcripts/SIWC Lesson 8.txt', [Summary(title='Lean Startup, Iteration Speed, and Building Better Tools with Eric Ries', summary='In this lesson, Eric Ries joins to discuss his reactions to SolveIT and shares insights from his experience developing the Lean Startup methodology. Key themes include:\n\n1. The importance of rapid iteration and feedback loops in product development\n- Smaller batch sizes and faster feedback cycles lead to better outcomes\n- The build-measure-learn feedback loop is fundamental to learning and improvement\n- Technical tools that reduce friction enable faster iteration\n\n2. The value of being your own customer\n- Building tools you want to use yourself provides immediate feedback\n- Co-evolution between developers and tools leads to better products\n- Internal usage as a key metric of success\n\n3. The role of experimentation and scientific thinking\n- Treating assumptions as hypotheses that need testing\n- The importance of defining clear expectat

In [32]:
for t, s in zipped:
    print(t.split('/')[-1].replace('.txt', ''), s[0].title)

SIWC Lesson 8 Lean Startup, Iteration Speed, and Building Better Tools with Eric Ries
SIWC Lesson 9 Building with External AI APIs
SIWC - lesson1 Introduction to SolveIT and Dialogue Engineering
SIWC Lesson 11 Creating Web Apps with SolveIT and FastHTML
SIWC Lesson 10 Building an R&D Company with Values and Vision
SIWC Lesson 12 Building a Blog with Fast HTML and Monster UI
SIWC Lesson 2 SolveIT Lesson 2: Coding Process, List Comprehensions, and Problem Solving
SIWC Lesson 3 SolveIT Lesson 3: Beyond Advent of Code
SIWC Lesson 7 Web Scraping and Graph Problem Classification with Claude
SIWC Lesson 6 SolveIT Lesson: Building Reusable Code and Problem-Solving Techniques
SIWC Lesson 4 Shell Sage Demo and Learning Approaches
SIWC Lesson 5 SolveIT Lesson 5: Problem Solving Techniques and Tools
